In [1]:
# Operator type: (mode, type) where mode is :a, :b, :c, etc. and type is :creation or :annihilation
Base.@kwdef struct Operator
    mode::Symbol      # :a, :b, :c, etc.
    type::Symbol      # :creation or :annihilation
end

# Make Operator copyable and comparable
Base.copy(op::Operator) = Operator(mode=op.mode, type=op.type)
Base.:(==)(op1::Operator, op2::Operator) = op1.mode == op2.mode && op1.type == op2.type
Base.hash(op::Operator, h::UInt) = hash((op.mode, op.type), h)

# For cleaner notation
ad(mode::Symbol) = Operator(mode, :creation)
a(mode::Symbol) = Operator(mode, :annihilation)

# Check if operator is creation type
is_creation(op::Operator) = op.type == :creation
is_annihilation(op::Operator) = op.type == :annihilation

# Commutation relation: [a_i, a_j†] = δ_ij
# Returns (coefficient, swapped_ops) or nothing if can't commute
function commute(op1::Operator, op2::Operator)
    # Can only commute annihilation followed by creation
    if is_annihilation(op1) && is_creation(op2)
        if op1.mode == op2.mode
            # Same mode: [a, a†] = 1
            return (1, [op2, op1])
        else
            # Different modes: [a, b†] = 0, so just swap
            return (0, [op2, op1])
        end
    end
    return nothing
end

# Check if sequence is in normal order
function is_normal_ordered(ops::Vector{Operator})
    for i in 1:(length(ops)-1)
        if is_annihilation(ops[i]) && is_creation(ops[i+1])
            return false
        end
    end
    return true
end

# Main normal ordering function
function normal_order_full(ops::Vector{Operator}, debug=false)
    if length(ops) == 0
        return Dict(Operator[] => 1)
    end
    
    if is_normal_ordered(ops)
        return Dict([copy(op) for op in ops] => 1)
    end
    
    result = Dict{Vector{Operator}, Int}()
    
    # Find first pair that needs commuting (annihilation before creation)
    for i in 1:(length(ops)-1)
        if is_annihilation(ops[i]) && is_creation(ops[i+1])
            if debug
                println("Found pair at position $i: $(op_to_string(ops[i])) $(op_to_string(ops[i+1]))")
                println("  Modes: $(ops[i].mode) $(ops[i+1].mode), Types: $(ops[i].type) $(ops[i+1].type)")
                println("  Same mode? $(ops[i].mode == ops[i+1].mode)")
            end
            
            # Term 1: Swap the operators - create new copies
            ops_swapped = [copy(op) for op in vcat(ops[1:i-1], [ops[i+1], ops[i]], ops[i+2:end])]
            if debug
                println("  Swapped: ", join(op_to_string.(ops_swapped), " "))
                println("  Swapped details: ", [(op.mode, op.type) for op in ops_swapped])
            end
            sub_result1 = normal_order_full(ops_swapped, debug)
            for (key, val) in sub_result1
                result[key] = get(result, key, 0) + val
            end
            
            # Term 2: Commutator (only if same mode)
            if ops[i].mode == ops[i+1].mode
                ops_reduced = [copy(op) for op in vcat(ops[1:i-1], ops[i+2:end])]
                if debug
                    println("  Reduced: ", join(op_to_string.(ops_reduced), " "))
                    println("  Reduced details: ", [(op.mode, op.type) for op in ops_reduced])
                end
                sub_result2 = normal_order_full(ops_reduced, debug)
                for (key, val) in sub_result2
                    result[key] = get(result, key, 0) + val
                end
            end
            
            return result
        end
    end
    
    return Dict([copy(op) for op in ops] => 1)
end

# Pretty printing
function op_to_string(op::Operator)
    if is_creation(op)
        return "$(op.mode)†"
    else
        return "$(op.mode)"
    end
end

function print_normal_order(ops::Vector{Operator}, debug=false)
    println("\nStarting with: ", join(op_to_string.(ops), " "))
    result = normal_order_full(ops, debug)
    
    println("Normal-ordered form:")
    
    sorted_keys = sort(collect(keys(result)), by = k -> (length(k), join(op_to_string.(k))))
    
    terms = String[]
    for key in sorted_keys
        coef = result[key]
        if length(key) == 0
            if coef != 0
                push!(terms, string(coef))
            end
        else
            op_str = join(op_to_string.(key), " ")
            if coef == 1
                push!(terms, op_str)
            elseif coef == -1
                push!(terms, "-$op_str")
            else
                push!(terms, "$coef($op_str)")
            end
        end
    end
    
    if length(terms) == 0
        println("  = 0")
    else
        println("  = ", join(terms, " + "))
    end
end


print_normal_order (generic function with 2 methods)

In [129]:
using AbstractAlgebra

# Define non-commutative algebra
R, (α, αstar, dα, dαstar, W) = free_associative_algebra(QQ, ["α", "α*", "∂α", "∂α*", "W"])

# Operator definitions
Base.@kwdef struct Operator
    mode::Symbol
    type::Symbol
end

Base.copy(op::Operator) = Operator(mode=op.mode, type=op.type)
Base.:(==)(op1::Operator, op2::Operator) = op1.mode == op2.mode && op1.type == op2.type
Base.hash(op::Operator, h::UInt) = hash((op.mode, op.type), h)

ad(mode::Symbol) = Operator(mode, :creation)
a(mode::Symbol) = Operator(mode, :annihilation)

is_creation(op::Operator) = op.type == :creation
is_annihilation(op::Operator) = op.type == :annihilation

# Wigner variable type
Base.@kwdef struct WignerVar
    variable::Symbol  # :α or :αstar
    type::Symbol      # :variable or :derivative
end

Base.copy(wv::WignerVar) = WignerVar(variable=wv.variable, type=wv.type)
Base.:(==)(wv1::WignerVar, wv2::WignerVar) = wv1.variable == wv2.variable && wv1.type == wv2.type
Base.hash(wv::WignerVar, h::UInt) = hash((wv.variable, wv.type), h)

# Helper constructors
wv_α() = WignerVar(:α, :variable)
wv_αstar() = WignerVar(:αstar, :variable)
wv_dα() = WignerVar(:α, :derivative)
wv_dαstar() = WignerVar(:αstar, :derivative)

# Pretty printing
function Base.show(io::IO, wv::WignerVar)
    if wv.type == :derivative
        print(io, "∂", wv.variable)
    else
        print(io, wv.variable)
    end
end

# Transform operator*rho (rho on right)
function op_rho_to_wigner(op::Operator)
    """
    Transform operator*rho to Wigner form (no W)
    ad*rho -> α* - 1/2 ∂α
    a*rho -> α + 1/2 ∂α*
    """
    if is_creation(op)
        return αstar - R(1//2) * dα
    else
        return α + R(1//2) * dαstar
    end
end

# Transform rho*operator (rho on left)
function rho_op_to_wigner(op::Operator)
    """
    Transform rho*operator to Wigner form (no W)
    rho*ad -> α* + 1/2 ∂α
    rho*a -> α - 1/2 ∂α*
    """
    if is_creation(op)
        return αstar + R(1//2) * dα
    else
        return α - R(1//2) * dαstar
    end
end

# Transform with rho position specified
function operators_to_wigner(ops::Vector{Operator}, rho_position::Symbol=:right)
    """
    Transform operators with rho
    
    rho_position:
      :right -> [op1, op2]*rho = (op1*rho) * (op2*rho) * ...
      :left  -> rho*[op1, op2] = (rho*op1) * (rho*op2) * ...
    """
    
    if length(ops) == 0
        return R(1)
    end
    
    # Select transformation based on rho position
    transform_func = rho_position == :right ? op_rho_to_wigner : rho_op_to_wigner
    
    # Build product
    result = transform_func(ops[1])
    
    for i in 2:length(ops)
        result = result * transform_func(ops[i])
    end
    
    return result
end

# Parse monomial string to WignerVar vector
function parse_monomial_to_wignervars(s::String)
    """Parse monomial string into vector of WignerVar"""
    result = WignerVar[]
    
    # Use character-aware iteration
    chars = collect(s)
    i = 1
    
    while i <= length(chars)
        # Check for ∂α* 
        if i + 2 <= length(chars) && chars[i] == '∂' && chars[i+1] == 'α' && chars[i+2] == 't'
            push!(result, wv_dαstar())
            i += 3
        # Check for ∂α
        elseif i + 1 <= length(chars) && chars[i] == '∂' && chars[i+1] == 'α'
            push!(result, wv_dα())
            i += 2
        # Check for α*
        elseif i + 1 <= length(chars) && chars[i] == 'α' && chars[i+1] == 't'
            push!(result, wv_αstar())
            i += 2
        # Check for α
        elseif chars[i] == 'α'
            push!(result, wv_α())
            i += 1
        else
            i += 1  # Skip unknown
        end
    end
    
    return result
end

# Get terms with coefficients as WignerVar vectors
function get_terms_with_coeffs(expr)
    result = []
    
    for (coeff, mon) in zip(coefficients(expr), monomials(expr))
        s = string(mon)
        s_clean = replace(s, " " => "")
        
        # Parse to WignerVar vector
        term = parse_monomial_to_wignervars(s_clean)
        
        push!(result, (coeff, term))
    end
    
    return result
end


get_terms_with_coeffs (generic function with 1 method)

In [268]:
using AbstractAlgebra

# Define non-commutative algebra
R, (α, αstar, dα, dαstar, W) = free_associative_algebra(QQ, ["α", "αt", "∂α", "∂αt", "W"])

# Operator definitions
Base.@kwdef struct Operator
    mode::Symbol
    type::Symbol
end

Base.copy(op::Operator) = Operator(mode=op.mode, type=op.type)
Base.:(==)(op1::Operator, op2::Operator) = op1.mode == op2.mode && op1.type == op2.type
Base.hash(op::Operator, h::UInt) = hash((op.mode, op.type), h)

ad(mode::Symbol) = Operator(mode, :creation)
a(mode::Symbol) = Operator(mode, :annihilation)

is_creation(op::Operator) = op.type == :creation
is_annihilation(op::Operator) = op.type == :annihilation

# Wigner variable type
Base.@kwdef struct WignerVar
    variable::Symbol  # :α or :αstar
    type::Symbol      # :variable or :derivative
end

Base.copy(wv::WignerVar) = WignerVar(variable=wv.variable, type=wv.type)
Base.:(==)(wv1::WignerVar, wv2::WignerVar) = wv1.variable == wv2.variable && wv1.type == wv2.type
Base.hash(wv::WignerVar, h::UInt) = hash((wv.variable, wv.type), h)

# Helper constructors
wv_α() = WignerVar(:α, :variable)
wv_αstar() = WignerVar(:αstar, :variable)
wv_dα() = WignerVar(:α, :derivative)
wv_dαstar() = WignerVar(:αstar, :derivative)

# Pretty printing
function Base.show(io::IO, wv::WignerVar)
    if wv.type == :derivative
        print(io, "∂", wv.variable)
    else
        print(io, wv.variable)
    end
end

# Transform operator*rho (rho on right)
function op_rho_to_wigner(op::Operator)
    """
    Transform operator*rho to Wigner form (no W)
    ad*rho -> α* - 1/2 ∂α
    a*rho -> α + 1/2 ∂α*
    """
    if is_creation(op)
        return αstar - R(1//2) * dα
    else
        return α + R(1//2) * dαstar
    end
end

# Transform rho*operator (rho on left)
function rho_op_to_wigner(op::Operator)
    """
    Transform rho*operator to Wigner form (no W)
    rho*ad -> α* + 1/2 ∂α
    rho*a -> α - 1/2 ∂α*
    """
    if is_creation(op)
        return αstar + R(1//2) * dα
    else
        return α - R(1//2) * dαstar
    end
end

# Transform with rho position specified
function operators_to_wigner(ops::Vector{Operator}, rho_position::Symbol=:right)
    """
    Transform operators with rho
    
    rho_position:
      :right -> [op1, op2]*rho = (op1*rho) * (op2*rho) * ...
      :left  -> rho*[op1, op2] = (rho*op1) * (rho*op2) * ...
    """
    
    if length(ops) == 0
        return R(1)
    end
    
    # Select transformation based on rho position
    transform_func = rho_position == :right ? op_rho_to_wigner : rho_op_to_wigner
    
    # Build product
    result = transform_func(ops[1])
    
    for i in 2:length(ops)
        result = result * transform_func(ops[i])
    end
    
    return result
end

function parse_monomial_to_wignervars(s::String)
    """Parse monomial string into vector of WignerVar"""
    result = WignerVar[]
    
    # Use character-aware iteration
    chars = collect(s)
    i = 1
    
    while i <= length(chars)
        var_to_add = nothing
        exponent = 1
        chars_consumed = 0
        
        # Check for ∂α* 
        if i + 2 <= length(chars) && chars[i] == '∂' && chars[i+1] == 'α' && chars[i+2] == 't'
            var_to_add = wv_dαstar()
            chars_consumed = 3
        # Check for ∂α
        elseif i + 1 <= length(chars) && chars[i] == '∂' && chars[i+1] == 'α'
            var_to_add = wv_dα()
            chars_consumed = 2
        # Check for αt (α*)
        elseif i + 1 <= length(chars) && chars[i] == 'α' && chars[i+1] == 't'
            var_to_add = wv_αstar()
            chars_consumed = 2
        # Check for α
        elseif chars[i] == 'α'
            var_to_add = wv_α()
            chars_consumed = 1
        else
            i += 1  # Skip unknown
            continue
        end
        
        i += chars_consumed
        
        # Check for exponent (^number)
        if i <= length(chars) && chars[i] == '^'
            i += 1  # Skip ^
            exp_str = ""
            while i <= length(chars) && isdigit(chars[i])
                exp_str *= chars[i]
                i += 1
            end
            if !isempty(exp_str)
                exponent = parse(Int, exp_str)
            end
        end
        
        # Add the variable 'exponent' times
        for _ in 1:exponent
            push!(result, var_to_add)
        end
    end
    
    return result
end

# Get terms with coefficients as WignerVar vectors
function get_terms_with_coeffs(expr)
    result = []
    
    for (coeff, mon) in zip(coefficients(expr), monomials(expr))
        s = string(mon)
        s_clean = replace(s, " " => "")
        
        # Parse to WignerVar vector
        term = parse_monomial_to_wignervars(s_clean)
        
        push!(result, (coeff, term))
    end
    
    return result
end



function wigner_to_symbolic(wv::WignerVar)
    if wv.type == :variable
        if wv.variable == :α
            return α_sym
        elseif wv.variable == :αstar
            return αstar_sym
        end
    elseif wv.type == :derivative
        if wv.variable == :α
            return dα_sym
        elseif wv.variable == :αstar
            return dαstar_sym
        end
    end
    error("Unknown WignerVar: $wv")
end

function symbolic_to_wigner(sym)
    if sym == α_sym
        return wv_α()
    elseif sym == αstar_sym
        return wv_αstar()
    elseif sym == dα_sym
        return wv_dα()
    elseif sym == dαstar_sym
        return wv_dαstar()
    else
        error("Unknown symbolic variable: $sym")
    end
end


function symbolic_to_algebra(expr, var_map_reverse)
    # Unwrap Symbolics wrapper first
    expr_unwrapped = Symbolics.unwrap(expr)
    
    # Handle simple cases
    if haskey(var_map_reverse, expr)
        return var_map_reverse[expr]
    elseif isa(expr_unwrapped, Number)
        return R(Rational{BigInt}(expr_unwrapped))  # Convert to correct number type
    end
    
    # Handle operations
    if Symbolics.issym(expr_unwrapped)
        return var_map_reverse[expr]
    elseif Symbolics.isadd(expr_unwrapped)
        # Handle addition: a + b + c
        terms = Symbolics.arguments(expr_unwrapped)
        return sum(symbolic_to_algebra(t, var_map_reverse) for t in terms)
    elseif Symbolics.ismul(expr_unwrapped)
        # Handle multiplication: a * b * c
        factors = Symbolics.arguments(expr_unwrapped)
        result = R(1)  # Start with identity
        for f in factors
            result = result * symbolic_to_algebra(f, var_map_reverse)
        end
        return result
    elseif Symbolics.ispow(expr_unwrapped)
        # Handle power: a^n
        base_expr, exp_val = Symbolics.arguments(expr_unwrapped)
        base = symbolic_to_algebra(base_expr, var_map_reverse)
        return base^Int(exp_val)
    else
        error("Cannot convert symbolic expression: $expr of type $(typeof(expr_unwrapped))")
    end
end

symbolic_to_algebra (generic function with 1 method)

In [380]:
# For Operator type
is_derivative(op::Operator) = op.type == :derivative
is_variable(op::Operator) = op.type == :variable

# For WignerVar type
is_derivative(wv::WignerVar) = wv.type == :derivative
is_variable(wv::WignerVar) = wv.type == :variable

using Symbolics
@variables α_sym, αstar_sym, dα_sym, dαstar_sym, W_sym
var_map_sym = Dict(α => α_sym, αstar => αstar_sym, dα => dα_sym, dαstar => dαstar_sym, W => W_sym)


# Create reverse mapping from symbolic to algebra
var_map_reverse = Dict(
    α_sym => α,
    αstar_sym => αstar,
    dα_sym => dα,
    dαstar_sym => dαstar,
    W_sym => W
)


function is_derivative_ordered(ops::Vector{WignerVar})
    # Derivatives should all be on the left
    # So once we see a variable, we should never see a derivative after it
    seen_variable = false
    for op in ops
        if is_variable(op)
            seen_variable = true
        elseif is_derivative(op) && seen_variable
            return false  # Found a derivative after a variable - not ordered!
        end
    end
    return true
end

function derivative_ordering(ops_all::Tuple{Rational{BigInt}, Vector{WignerVar}}; debug=false)
    const_init, ops = ops_all
    
    if debug
        println("Called with: const=$const_init, ops=$(ops)")
    end
    
    if length(ops) == 0
        return Dict(WignerVar[] => const_init)
    end
    
    if is_derivative_ordered(ops)
        if debug
            println("Already ordered, returning")
        end
        return Dict([copy(op) for op in ops] => const_init)
    end

    result = Dict{Vector{WignerVar}, Rational{BigInt}}()
    
    # Find where derivatives end (last consecutive derivative from the left)
    deriv_end_idx = 0
    for i in 1:length(ops)
        if is_derivative(ops[i])
            deriv_end_idx = i
        else
            break
        end
    end
    
    # Find the FIRST variable-derivative pair that needs swapping
    for i in 1:(length(ops)-1)
        if is_variable(ops[i]) && is_derivative(ops[i+1])
            if debug
                println("Found swap at position $i: $(ops[i]) and $(ops[i+1])")
            end
            
            # Term 1: Swap the pair
            # Keep derivatives on left, swap i and i+1, keep rest
            left_derivs = ops[1:deriv_end_idx]
            middle = ops[deriv_end_idx+1:i-1]  # variables between last deriv and position i
            # Swap: move derivative to join left derivs
            ops_swapped = vcat(left_derivs, [ops[i+1]], middle, [ops[i]], ops[i+2:end])
            
            if debug
                println("  Swapped: $(ops_swapped)")
            end
            sub_result1 = derivative_ordering((const_init, ops_swapped); debug=debug)
            for (key, val) in sub_result1
                result[key] = get(result, key, Rational{BigInt}(0)) + val
                if debug
                    println("  Adding to result: $key => $val")
                end
            end

            # Term 2: Apply product rule
            # Variables from after last derivative up to and including position i
            vars_before = ops[deriv_end_idx+1:i]
            
            if debug
                println("  Variables before derivative: $(vars_before)")
            end
            
            if !isempty(vars_before)
                rev = prod(wigner_to_symbolic.(vars_before))
                
                if ops[i+1] == wv_dα() 
                    var_sym = α_sym
                elseif ops[i+1] == wv_dαstar() 
                    var_sym = αstar_sym
                else
                    error("Unknown derivative: $(ops[i+1])")
                end
                
                if debug
                    println("  Taking derivative of $rev with respect to $var_sym")
                end
                
                Dx = Differential(var_sym)
                result_dx = expand_derivatives(Dx(rev))
                
                if debug
                    println("  Derivative result: $result_dx")
                end
                
                if !iszero(result_dx)
                    algebra_expr = symbolic_to_algebra(result_dx, var_map_reverse)
                    terms_dx = get_terms_with_coeffs(algebra_expr)
                    
                    if debug
                        println("  Derivative terms: $terms_dx")
                    end
                    
                    # Process all terms from the derivative
                    for (constant_dx, ops_dx) in terms_dx
                        # Keep derivatives from left (derivative at i+1 is consumed)
                        ops_reduced = vcat(left_derivs, ops_dx, ops[i+2:end])
                        
                        if debug
                            println("  Reduced ops: const=$(const_init * constant_dx), ops=$(ops_reduced)")
                        end
                        
                        sub_result2 = derivative_ordering((const_init * constant_dx, ops_reduced); debug=debug)
                        for (key, val) in sub_result2
                            result[key] = get(result, key, Rational{BigInt}(0)) - val
                            if debug
                                println("  Subtracting from result: $key => $val")
                            end
                        end
                    end
                end
            end
            
            return result
        end
    end
    
    return Dict([copy(op) for op in ops] => const_init)
end

function derivative_ordering_commutator(ops_all::Tuple{Rational{BigInt}, Vector{WignerVar}}; debug=false)
    const_init, ops = ops_all
    
    if debug
        println("Called with: const=$const_init, ops=$(ops)")
    end
    
    if length(ops) == 0
        return Dict(WignerVar[] => const_init)
    end
    
    if is_derivative_ordered(ops)
        if debug
            println("Already ordered, returning")
        end
        return Dict([copy(op) for op in ops] => const_init)
    end

    result = Dict{Vector{WignerVar}, Rational{BigInt}}()
    
    # Find where derivatives end (last consecutive derivative from the left)
    deriv_end_idx = 0
    for i in 1:length(ops)
        if is_derivative(ops[i])
            deriv_end_idx = i
        else
            break
        end
    end
    
    # Find the FIRST variable-derivative pair that needs swapping
    for i in 1:(length(ops)-1)
        if is_variable(ops[i]) && is_derivative(ops[i+1])
            if debug
                println("Found swap at position $i: $(ops[i]) and $(ops[i+1])")
            end
            
            # Term 1: Swap the pair
            # Keep derivatives on left, swap i and i+1, keep rest
            left_derivs = ops[1:deriv_end_idx]
            middle = ops[deriv_end_idx+1:i-1]  # variables between last deriv and position i
            # Swap: move derivative to join left derivs
            ops_swapped = vcat(left_derivs, [ops[i+1]], middle, [ops[i]], ops[i+2:end])
            
            if debug
                println("  Swapped: $(ops_swapped)")
            end
            sub_result1 = derivative_ordering((const_init, ops_swapped); debug=debug)
            for (key, val) in sub_result1
                result[key] = get(result, key, Rational{BigInt}(0)) + val
                if debug
                    println("  Adding to result: $key => $val")
                end
            end

            # Term 2: Apply product rule
            # Variables from after last derivative up to and including position i
            vars_before = ops[deriv_end_idx+1:i]
            
            if debug
                println("  Variables before derivative: $(vars_before)")
            end
            
            if !isempty(vars_before)
                rev = prod(wigner_to_symbolic.(vars_before))
                
                if ops[i+1] == wv_dα() 
                    var_sym = α_sym
                elseif ops[i+1] == wv_dαstar() 
                    var_sym = αstar_sym
                else
                    error("Unknown derivative: $(ops[i+1])")
                end
                
                if debug
                    println("  Taking derivative of $rev with respect to $var_sym")
                end
                
                Dx = Differential(var_sym)
                result_dx = expand_derivatives(Dx(rev))
                
                if debug
                    println("  Derivative result: $result_dx")
                end
                
                if !iszero(result_dx)
                    algebra_expr = symbolic_to_algebra(result_dx, var_map_reverse)
                    terms_dx = get_terms_with_coeffs(algebra_expr)
                    
                    if debug
                        println("  Derivative terms: $terms_dx")
                    end
                    
                    # Process all terms from the derivative
                    for (constant_dx, ops_dx) in terms_dx
                        # Keep derivatives from left (derivative at i+1 is consumed)
                        ops_reduced = vcat(left_derivs,  ops[i+2:end])
                        
                        if debug
                            println("  Reduced ops: const=$(const_init * constant_dx), ops=$(ops_reduced)")
                        end
                        
                        sub_result2 = derivative_ordering((constant_dx, ops_reduced); debug=debug)
                        for (key, val) in sub_result2
                            result[key] = get(result, key, Rational{BigInt}(0)) - val
                            if debug
                                println("  Subtracting from result: $key => $val")
                            end
                        end
                    end
                end
            end
            
            return result
        end
    end
    
    return Dict([copy(op) for op in ops] => const_init)
end

function group_by_derivatives_symbolic(result::Dict{Vector{WignerVar}, Rational{BigInt}})
    grouped = Dict{Vector{WignerVar}, Any}()
    
    for (ops, coeff) in result
        # Split into derivatives and variables
        derivs = filter(is_derivative, ops)
        vars = filter(is_variable, ops)
        
        # Convert variables to symbolic expression
        if isempty(vars)
            var_expr = coeff
        else
            var_expr = coeff * prod(wigner_to_symbolic.(vars))
        end
        
        # Add to the group
        if !haskey(grouped, derivs)
            grouped[derivs] = var_expr
        else
            grouped[derivs] = grouped[derivs] + var_expr
        end
    end
    
    return grouped
end

group_by_derivatives_symbolic (generic function with 1 method)

In [395]:
function is_derivative_ordered_right(ops::Vector{WignerVar})
    # Derivatives should all be on the right
    # So once we see a derivative, we should never see a variable after it
    seen_derivative = false
    for op in ops
        if is_derivative(op)
            seen_derivative = true
        elseif is_variable(op) && seen_derivative
            return false  # Found a variable after a derivative - not ordered!
        end
    end
    return true
end

function derivative_ordering_right_com(ops_all::Tuple{Rational{BigInt}, Vector{WignerVar}}; debug=false)
    const_init, ops = ops_all
    
    if debug
        println("Called with: const=$const_init, ops=$(ops)")
    end
    
    if length(ops) == 0
        return Dict(WignerVar[] => const_init)
    end
    
    if is_derivative_ordered_right(ops)
        if debug
            println("Already ordered, returning")
        end
        return Dict([copy(op) for op in ops] => const_init)
    end

    result = Dict{Vector{WignerVar}, Rational{BigInt}}()
    
    # Find the FIRST derivative-variable pair that needs swapping
    for i in 1:(length(ops)-1)
        if is_derivative(ops[i]) && is_variable(ops[i+1])
            if debug
                println("Found swap at position $i: $(ops[i]) and $(ops[i+1])")
            end
            
            # Check if this is a matching pair for commutation rule
            is_matching_pair = (ops[i] == wv_dα() && ops[i+1] == wv_α()) ||
                              (ops[i] == wv_dαstar() && ops[i+1] == wv_αstar())
            
            # Term 1: Swap the adjacent pair
            ops_swapped = vcat(ops[1:i-1], [ops[i+1]], [ops[i]], ops[i+2:end])
            
            if debug
                println("  Swapped: $(ops_swapped)")
            end
            sub_result1 = derivative_ordering_right_com((const_init, ops_swapped); debug=debug)
            for (key, val) in sub_result1
                result[key] = get(result, key, Rational{BigInt}(0)) + val
            end

            # Term 2: Apply commutation rule (+1 for matching pairs)
            if is_matching_pair
                ops_reduced = vcat(ops[1:i-1], ops[i+2:end])
                
                if debug
                    println("  Commutation +1 term, reduced ops: $(ops_reduced)")
                end
                
                sub_result2 = derivative_ordering_right_com((const_init, ops_reduced); debug=debug)
                for (key, val) in sub_result2
                    result[key] = get(result, key, Rational{BigInt}(0)) + val
                end
            end
            
            return result
        end
    end
    
    return Dict([copy(op) for op in ops] => const_init)
end

function group_by_derivatives_symbolic_right(result::Dict{Vector{WignerVar}, Rational{BigInt}})
    grouped = Dict{Vector{WignerVar}, Any}()
    
    for (ops, coeff) in result
        # Find where variables end and derivatives begin
        # Since derivatives are on the right, we split at the boundary
        split_idx = length(ops)
        for i in 1:length(ops)
            if is_derivative(ops[i])
                split_idx = i - 1
                break
            end
        end
        
        # Split into variables (left) and derivatives (right)
        vars = ops[1:split_idx]
        derivs = ops[split_idx+1:end]
        
        # Convert variables to symbolic expression
        if isempty(vars)
            var_expr = coeff
        else
            var_expr = coeff * prod(wigner_to_symbolic.(vars))
        end
        
        # Add to the group with derivatives as the key
        if !haskey(grouped, derivs)
            grouped[derivs] = var_expr
        else
            grouped[derivs] = grouped[derivs] + var_expr
        end
    end
    
    return grouped
end

group_by_derivatives_symbolic_right (generic function with 1 method)

In [341]:
 function derivative_ordering_commutator(ops_all::Tuple{Rational{BigInt}, Vector{WignerVar}}; debug=false)
    const_init, ops = ops_all
    
    if debug
        println("Called with: const=$const_init, ops=$(ops)")
    end
    
    if length(ops) == 0
        return Dict(WignerVar[] => const_init)
    end
    
    if is_derivative_ordered(ops)
        if debug
            println("Already ordered, returning")
        end
        return Dict([copy(op) for op in ops] => const_init)
    end

    result = Dict{Vector{WignerVar}, Rational{BigInt}}()
    
    # Find first pair that needs commuting (variable before derivative)
    for i in 1:(length(ops)-1)
        if is_variable(ops[i]) && is_derivative(ops[i+1])
            if debug
                println("Found pair at position $i: $(ops[i]) $(ops[i+1])")
                println("  Variables: $(ops[i].variable) $(ops[i+1].variable)")
                println("  Same variable? $(ops[i].variable == ops[i+1].variable)")
            end
            
            # Term 1: Swap the operators - create new copies
            ops_swapped = [copy(op) for op in vcat(ops[1:i-1], [ops[i+1], ops[i]], ops[i+2:end])]
            if debug
                println("  Swapped: $(ops_swapped)")
            end
            sub_result1 = derivative_ordering_commutator((const_init, ops_swapped); debug=debug)
            for (key, val) in sub_result1
                result[key] = get(result, key, Rational{BigInt}(0)) + val
                if debug
                    println("  Adding swapped term: $key => $val")
                end
            end
            
            # Term 2: Commutator (only if same variable: α with ∂α or α* with ∂α*)
            if ops[i].variable == ops[i+1].variable
                # [variable, ∂variable] = -1, so we subtract
                ops_reduced = [copy(op) for op in vcat(ops[1:i-1], ops[i+2:end])]
                if debug
                    println("  Commutator: -1")
                    println("  Reduced: $(ops_reduced)")
                end
                sub_result2 = derivative_ordering_commutator((-const_init, ops_reduced); debug=debug)
                for (key, val) in sub_result2
                    result[key] = get(result, key, Rational{BigInt}(0)) + val
                    if debug
                        println("  Adding commutator term: $key => $val")
                    end
                end
            end
            # If different variables (e.g., α with ∂α*), commutator is zero
            
            return result
        end
    end
    
    return Dict([copy(op) for op in ops] => const_init)
end

derivative_ordering_commutator (generic function with 1 method)

In [307]:
terms1[3]

(1//2, WignerVar[α, α, ∂α, αstar])

In [310]:
terms1[3][2][3].variable==terms1[3][2][4].variable

false

In [347]:
# First term of D[a]
ops = [a(:a),a(:a)]
result01 = operators_to_wigner(ops, :right)
result02 = operators_to_wigner([ad(:a), ad(:a)], :left)
result1 = result01*result02
terms1 = get_terms_with_coeffs(result1);

result_1st = Dict{Vector{WignerVar}, Rational{BigInt}}()
for term in terms1
    for (ops, coeff) in derivative_ordering_commutator(term)
        result_1st[ops] = get(result_1st, ops, Rational{BigInt}(0)) + coeff
    end
end


# Second term
result2 = operators_to_wigner([ad(:a),ad(:a), a(:a), a(:a)], :right)*(-1/2)
terms2 = get_terms_with_coeffs(result2);
result_2nd = Dict{Vector{WignerVar}, Rational{BigInt}}()

for term in terms2
    for (ops, coeff) in derivative_ordering_commutator(term)
        result_2nd[ops] = get(result_2nd, ops, Rational{BigInt}(0)) + coeff
    end
end


# Third term
result3 = operators_to_wigner([ad(:a),ad(:a), a(:a), a(:a)], :left)*(-1/2)
terms3 = get_terms_with_coeffs(result3);
result_3rd = Dict{Vector{WignerVar}, Rational{BigInt}}()

for term in terms3
    for (ops, coeff) in derivative_ordering_commutator(term)
        result_3rd[ops] = get(result_3rd, ops, Rational{BigInt}(0)) + coeff
    end
end

# Combine all three results by adding coefficients
result_merge = copy(result_1st)
for (ops, coeff) in result_2nd
    result_merge[ops] = get(result_merge, ops, Rational{BigInt}(0)) + coeff
end
for (ops, coeff) in result_3rd
    result_merge[ops] = get(result_merge, ops, Rational{BigInt}(0)) + coeff
end

group_all = group_by_derivatives_symbolic(result_merge);

In [314]:
# First term of D[a]
ops = [a(:a)]
result01 = operators_to_wigner(ops, :right)
result02 = operators_to_wigner([ad(:a)], :left)
result1 = result01*result02
terms1 = get_terms_with_coeffs(result1);

result_1st = Dict{Vector{WignerVar}, Rational{BigInt}}()
for term in terms1
    merge!(+, result_1st , derivative_ordering(term))
end


# Second term
result2 = operators_to_wigner([a(:a),  ad(:a)], :right)*(-1/2)
terms2 = get_terms_with_coeffs(result2);
result_2nd = Dict{Vector{WignerVar}, Rational{BigInt}}()

for term in terms2
    merge!(+, result_2nd, derivative_ordering(term))
end


# Third term
result3 = operators_to_wigner([ad(:a),  a(:a)], :left)*(-1/2)
terms3 = get_terms_with_coeffs(result3);
result_3rd = Dict{Vector{WignerVar}, Rational{BigInt}}()

for term in terms3
    merge!(+, result_3rd, derivative_ordering(term))
end

result_merge = merge(result_1st, result_2nd, result_3rd)
group_all = group_by_derivatives_symbolic(result_merge);


In [153]:
# Second term
result2 = operators_to_wigner([ad(:a),  a(:a)], :right)*(-1/2)
terms2 = get_terms_with_coeffs(result2);
resultn = Dict{Vector{WignerVar}, Rational{BigInt}}()

for term in terms2
    merge!(+, result, derivative_ordering(term))
    merge!(+, resultn, derivative_ordering(term))
end


resultn

Dict{Vector{WignerVar}, Rational{BigInt}} with 5 entries:
  []              => 1//4
  [αstar, α]      => -1//2
  [∂α, α]         => 1//4
  [∂αstar, αstar] => -1//4
  [∂α, ∂αstar]    => 1//8

In [323]:
# First term of D[a]
ops = [a(:a)]
result01 = operators_to_wigner(ops, :right)
result02 = operators_to_wigner([ad(:a)], :left)
result1 = result01*result02
terms1 = get_terms_with_coeffs(result1);

result_1st = Dict{Vector{WignerVar}, Rational{BigInt}}()
for term in terms1
    for (ops, coeff) in derivative_ordering_commutator(term)
        result_1st[ops] = get(result_1st, ops, Rational{BigInt}(0)) + coeff
    end
end


# Second term
result2 = operators_to_wigner([ad(:a), a(:a)], :right)*(-1/2)
terms2 = get_terms_with_coeffs(result2);
result_2nd = Dict{Vector{WignerVar}, Rational{BigInt}}()

for term in terms2
    for (ops, coeff) in derivative_ordering_commutator(term)
        result_2nd[ops] = get(result_2nd, ops, Rational{BigInt}(0)) + coeff
    end
end


# Third term
result3 = operators_to_wigner([ad(:a), a(:a)], :left)*(-1/2)
terms3 = get_terms_with_coeffs(result3);
result_3rd = Dict{Vector{WignerVar}, Rational{BigInt}}()

for term in terms3
    for (ops, coeff) in derivative_ordering_commutator(term)
        result_3rd[ops] = get(result_3rd, ops, Rational{BigInt}(0)) + coeff
    end
end

# Combine all three results by adding coefficients
result_merge = copy(result_1st)
for (ops, coeff) in result_2nd
    result_merge[ops] = get(result_merge, ops, Rational{BigInt}(0)) + coeff
end
for (ops, coeff) in result_3rd
    result_merge[ops] = get(result_merge, ops, Rational{BigInt}(0)) + coeff
end

group_all = group_by_derivatives_symbolic(result_merge);

In [387]:
# First term of D[a]
ops = [a(:a)]
result01 = operators_to_wigner(ops, :right)
result02 = operators_to_wigner([ad(:a)], :left)
result1 = result01*result02
terms1 = get_terms_with_coeffs(result1);

result_1st = Dict{Vector{WignerVar}, Rational{BigInt}}()
for term in terms1
    for (ops, coeff) in derivative_ordering_right_com(term)
        result_1st[ops] = get(result_1st, ops, Rational{BigInt}(0)) + coeff
    end
end


# Second term
result2 = operators_to_wigner([ad(:a), a(:a)], :right)*(-1/2)
terms2 = get_terms_with_coeffs(result2);
result_2nd = Dict{Vector{WignerVar}, Rational{BigInt}}()

for term in terms2
    for (ops, coeff) in derivative_ordering_right_com(term)
        result_2nd[ops] = get(result_2nd, ops, Rational{BigInt}(0)) + coeff
    end
end


# Third term
result3 = operators_to_wigner([ad(:a), a(:a)], :left)*(-1/2)
terms3 = get_terms_with_coeffs(result3);
result_3rd = Dict{Vector{WignerVar}, Rational{BigInt}}()

for term in terms3
    for (ops, coeff) in derivative_ordering_right_com(term)
        result_3rd[ops] = get(result_3rd, ops, Rational{BigInt}(0)) + coeff
    end
end

# Combine all three results by adding coefficients
result_merge = copy(result_1st)
for (ops, coeff) in result_2nd
    result_merge[ops] = get(result_merge, ops, Rational{BigInt}(0)) + coeff
end
for (ops, coeff) in result_3rd
    result_merge[ops] = get(result_merge, ops, Rational{BigInt}(0)) + coeff
end

terms_arranged = Vector{Tuple{Rational{BigInt}, Vector{WignerVar}}}()
for (ops, coeff) in result_merge
    push!(terms_arranged, (coeff, ops))
end

result_all = Dict{Vector{WignerVar}, Rational{BigInt}}()

for term in terms_arranged
    for (ops, coeff) in derivative_ordering(term)
        result_all[ops] = get(result_all, ops, Rational{BigInt}(0)) + coeff
    end
end

group_all = group_by_derivatives_symbolic(result_all);

In [414]:
# First term of D[a]
ops = [a(:a)]
result01 = operators_to_wigner(ops, :right)
result02 = operators_to_wigner([ad(:a)], :left)
result1 = result01*result02
terms1 = get_terms_with_coeffs(result1);

result_1st = Dict{Vector{WignerVar}, Rational{BigInt}}()
for term in terms1
    for (ops, coeff) in derivative_ordering_right_com(term)
        result_1st[ops] = get(result_1st, ops, Rational{BigInt}(0)) + coeff
    end
end


# Second term
result2 = operators_to_wigner([ad(:a), a(:a)], :right)*(-1/2)
terms2 = get_terms_with_coeffs(result2);
result_2nd = Dict{Vector{WignerVar}, Rational{BigInt}}()

for term in terms2
    for (ops, coeff) in derivative_ordering_right_com(term)
        result_2nd[ops] = get(result_2nd, ops, Rational{BigInt}(0)) + coeff
    end
end


# Third term
result3 = operators_to_wigner([a(:a), ad(:a)], :left)*(-1/2)
terms3 = get_terms_with_coeffs(result3);
result_3rd = Dict{Vector{WignerVar}, Rational{BigInt}}()

for term in terms3
    for (ops, coeff) in derivative_ordering_right_com(term)
        result_3rd[ops] = get(result_3rd, ops, Rational{BigInt}(0)) + coeff
    end
end

# Combine all three results by adding coefficients
result_merge = copy(result_1st)
for (ops, coeff) in result_2nd
    result_merge[ops] = get(result_merge, ops, Rational{BigInt}(0)) + coeff
end
for (ops, coeff) in result_3rd
    result_merge[ops] = get(result_merge, ops, Rational{BigInt}(0)) + coeff
end

terms_arranged = Vector{Tuple{Rational{BigInt}, Vector{WignerVar}}}()
for (ops, coeff) in result_merge
    push!(terms_arranged, (coeff, ops))
end

result_all = Dict{Vector{WignerVar}, Rational{BigInt}}()

for term in terms_arranged
    for (ops, coeff) in derivative_ordering(term)
        result_all[ops] = get(result_all, ops, Rational{BigInt}(0)) + coeff
    end
end

group_all = group_by_derivatives_symbolic(result_all);

In [422]:
# First term of D[a]
ops = [a(:a),a(:a)]
result01 = operators_to_wigner(ops, :right)
result02 = operators_to_wigner([ad(:a),ad(:a)], :left)
result1 = result01*result02
terms1 = get_terms_with_coeffs(result1);

result_1st = Dict{Vector{WignerVar}, Rational{BigInt}}()
for term in terms1
    for (ops, coeff) in derivative_ordering_right_com(term)
        result_1st[ops] = get(result_1st, ops, Rational{BigInt}(0)) + coeff
    end
end


# Second term
result2 = operators_to_wigner([ad(:a),ad(:a), a(:a), a(:a)], :right)*(-1/2)
terms2 = get_terms_with_coeffs(result2);
result_2nd = Dict{Vector{WignerVar}, Rational{BigInt}}()

for term in terms2
    for (ops, coeff) in derivative_ordering_right_com(term)
        result_2nd[ops] = get(result_2nd, ops, Rational{BigInt}(0)) + coeff
    end
end


# Third term
result3 = operators_to_wigner([a(:a), a(:a), ad(:a), ad(:a)], :left)*(-1/2)
terms3 = get_terms_with_coeffs(result3);
result_3rd = Dict{Vector{WignerVar}, Rational{BigInt}}()

for term in terms3
    for (ops, coeff) in derivative_ordering_right_com(term)
        result_3rd[ops] = get(result_3rd, ops, Rational{BigInt}(0)) + coeff
    end
end

# Combine all three results by adding coefficients
result_merge = copy(result_1st)
for (ops, coeff) in result_2nd
    result_merge[ops] = get(result_merge, ops, Rational{BigInt}(0)) + coeff
end
for (ops, coeff) in result_3rd
    result_merge[ops] = get(result_merge, ops, Rational{BigInt}(0)) + coeff
end

terms_arranged = Vector{Tuple{Rational{BigInt}, Vector{WignerVar}}}()
for (ops, coeff) in result_merge
    push!(terms_arranged, (coeff, ops))
end

result_all = Dict{Vector{WignerVar}, Rational{BigInt}}()

for term in terms_arranged
    for (ops, coeff) in derivative_ordering(term)
        result_all[ops] = get(result_all, ops, Rational{BigInt}(0)) + coeff
    end
end

group_all = group_by_derivatives_symbolic(result_all);

In [407]:
derivative_ordering(terms_arranged[7])

Dict{Vector{WignerVar}, Rational{BigInt}} with 1 entry:
  [∂α, ∂αstar] => 1//4

In [403]:
derivative_ordering(terms_arranged[2])

Dict{Vector{WignerVar}, Rational{BigInt}} with 1 entry:
  [] => 1//2

In [416]:
result_1st

Dict{Vector{WignerVar}, Rational{BigInt}} with 5 entries:
  [∂αstar, ∂α]    => 1//4
  []              => 1//2
  [αstar, ∂αstar] => 1//2
  [α, αstar]      => 1
  [α, ∂α]         => 1//2

In [430]:
group_Da2

Dict{Vector{WignerVar}, Any} with 13 entries:
  [∂α, ∂α]                 => 0//1
  [∂αstar, ∂αstar]         => 0//1
  [∂αstar]                 => -αstar_sym + α_sym*(αstar_sym^2)
  [∂α, ∂α, ∂αstar]         => (-1//8)*α_sym
  [∂αstar, ∂α, ∂α]         => (3//8)*α_sym
  [∂α]                     => -α_sym + (α_sym^2)*αstar_sym
  [∂αstar, ∂αstar, ∂α]     => (1//8)*αstar_sym
  [∂α, ∂αstar, ∂αstar]     => (1//8)*αstar_sym
  [∂αstar, ∂α]             => -(3//4) + (3//2)*α_sym*αstar_sym
  []                       => 0
  [∂αstar, ∂αstar, ∂α, ∂α] => 1//32
  [∂α, ∂α, ∂αstar, ∂αstar] => -1//32
  [∂α, ∂αstar]             => -(1//4) + (1//2)*α_sym*αstar_sym

In [505]:
function subtract_wigner_dicts(dict1::Dict{Vector{WignerVar}, Any}, dict2::Dict{Vector{WignerVar}, Any}; fac=1)
    """
    Compute dict1 - dict2 for Wigner dictionaries with symbolic coefficients
    """
    result = Dict{Vector{WignerVar}, Any}()
    
    # Get all unique keys from both dictionaries
    all_keys = union(keys(dict1), keys(dict2))
    
    for key in all_keys
        val1 = get(dict1, key, 0)
        val2 = get(dict2, key, 0)
        
        # Subtract symbolically
        diff = Symbolics.simplify(val1 - fac*val2)
        
        # Only keep non-zero terms
        if !iszero(diff)
            result[key] = diff
        end
    end
    
    return result
end
function symmetrize_derivatives(dict::Dict{Vector{WignerVar}, Any})
    """
    Symmetrize derivative terms by grouping all permutations of the same derivatives
    and averaging their coefficients.
    
    For example: [∂α, ∂α, ∂αstar] and [∂αstar, ∂α, ∂α] and [∂α, ∂αstar, ∂α] 
    all represent the same mixed derivative ∂²α ∂αstar and should be combined.
    """
    result = Dict{Vector{WignerVar}, Any}()
    processed = Set{Vector{WignerVar}}()
    
    for (ops, coeff) in dict
        # Skip if we've already processed this or a permutation of it
        if ops in processed
            continue
        end
        
        # Get canonical form (sorted)
        canonical_ops = sort(ops, by = op -> (op.type, op.variable))
        
        # Find all permutations of this derivative set in the original dict
        matching_keys = Vector{Vector{WignerVar}}()
        matching_coeffs = Any[]
        
        for (other_ops, other_coeff) in dict
            # Check if this is a permutation of the same derivatives
            if sort(other_ops, by = op -> (op.type, op.variable)) == canonical_ops
                push!(matching_keys, other_ops)
                push!(matching_coeffs, other_coeff)
                push!(processed, other_ops)
            end
        end
        
        # Average the coefficients
        n_perms = length(matching_coeffs)
        avg_coeff = sum(matching_coeffs) / n_perms
        avg_coeff = Symbolics.simplify(avg_coeff)
        
        if !iszero(avg_coeff)
            result[canonical_ops] = avg_coeff
        end
    end
    
    return result
end

# Usage:
aval1 = fokker_planck_Heisenberg([ad(:a), ad(:a), ad(:a), a(:a)])
aval2 = fokker_planck_Heisenberg([ad(:a), a(:a), a(:a), a(:a)])

# Symmetrize each before subtracting
aval1_sym = symmetrize_derivatives(aval1)
aval2_sym = symmetrize_derivatives(aval2)

# Now subtract
result_diff = subtract_wigner_dicts(aval1_sym, aval2_sym)

Dict{Vector{WignerVar}, Any} with 6 entries:
  [∂α, ∂α, ∂α]             => (-1//4)*α_sym
  [∂αstar]                 => (3//1)*α_sym - (3//1)*(α_sym^2)*αstar_sym + αstar…
  [∂α, ∂αstar, ∂αstar]     => (3//8)*α_sym
  [∂α, ∂α, ∂αstar]         => (3//8)*αstar_sym
  [∂αstar, ∂αstar, ∂αstar] => (-1//4)*αstar_sym
  [∂α]                     => (3//1)*αstar_sym + α_sym^3 - (3//1)*α_sym*(αstar_…

In [460]:
result_diff[[wv_dαstar()]]

(3//1)*α_sym - (3//1)*(α_sym^2)*αstar_sym + αstar_sym^3

In [453]:
aval1 = fokker_planck_Heisenberg([a(:a), a(:a), a(:a), a(:a)])
aval2 = fokker_planck_Heisenberg([ad(:a), ad(:a), ad(:a), ad(:a)])

# Symmetrize each before subtracting
aval1_sym = symmetrize_derivatives(aval1)
aval2_sym = symmetrize_derivatives(aval2)

# Now subtract
result_diff = subtract_wigner_dicts(aval1_sym, aval2_sym)

Dict{Vector{WignerVar}, Any} with 4 entries:
  [∂α, ∂α, ∂α]             => αstar_sym
  [∂αstar]                 => (4//1)*(α_sym^3)
  [∂αstar, ∂αstar, ∂αstar] => α_sym
  [∂α]                     => (4//1)*(αstar_sym^3)

In [498]:
aval1 = fokker_planck_Heisenberg([ad(:a), ad(:a), ad(:a), a(:a)])
aval2 = fokker_planck_Heisenberg([ad(:a), a(:a), a(:a), a(:a)])

# Symmetrize each before subtracting
aval1_sym = symmetrize_derivatives(aval1)
aval2_sym = symmetrize_derivatives(aval2)

# Now subtract
result_diff = subtract_wigner_dicts(aval1_sym, aval2_sym)

Dict{Vector{WignerVar}, Any} with 6 entries:
  [∂α, ∂α, ∂α]             => (-1//4)*α_sym
  [∂αstar]                 => (3//1)*α_sym - (3//1)*(α_sym^2)*αstar_sym + αstar…
  [∂α, ∂αstar, ∂αstar]     => (3//8)*α_sym
  [∂α, ∂α, ∂αstar]         => (3//8)*αstar_sym
  [∂αstar, ∂αstar, ∂αstar] => (-1//4)*αstar_sym
  [∂α]                     => (3//1)*αstar_sym + α_sym^3 - (3//1)*α_sym*(αstar_…

In [508]:
aval1 = fokker_planck_Heisenberg([ad(:a), ad(:a), ad(:a), a(:a)])
aval2 = fokker_planck_Heisenberg([ad(:a), a(:a), a(:a), a(:a)])

# Symmetrize each before subtracting
aval1_sym = symmetrize_derivatives(aval1)
aval2_sym = symmetrize_derivatives(aval2)

# Now subtract
result_diff = subtract_wigner_dicts(aval1_sym, aval2_sym; fac=-1)

Dict{Vector{WignerVar}, Any} with 6 entries:
  [∂α, ∂α, ∂α]             => (-1//4)*α_sym
  [∂αstar]                 => -(3//1)*α_sym + (3//1)*(α_sym^2)*αstar_sym + αsta…
  [∂α, ∂αstar, ∂αstar]     => (-3//8)*α_sym
  [∂α, ∂α, ∂αstar]         => (3//8)*αstar_sym
  [∂αstar, ∂αstar, ∂αstar] => (1//4)*αstar_sym
  [∂α]                     => (3//1)*αstar_sym - (α_sym^3) - (3//1)*α_sym*(αsta…

In [510]:
result_diff[[wv_dα()]]

(3//1)*αstar_sym - (α_sym^3) - (3//1)*α_sym*(αstar_sym^2)

In [454]:
aval1 = fokker_planck_Heisenberg([a(:a), a(:a)])
aval2 = fokker_planck_Heisenberg([ad(:a), ad(:a)])

# Symmetrize each before subtracting
aval1_sym = symmetrize_derivatives(aval1)
aval2_sym = symmetrize_derivatives(aval2)

# Now subtract
result_diff = subtract_wigner_dicts(aval1_sym, aval2_sym)

Dict{Vector{WignerVar}, Any} with 2 entries:
  [∂αstar] => (2//1)*α_sym
  [∂α]     => (2//1)*αstar_sym

In [462]:
result = fokker_planck_dissipator([a(:a)])
aval1_sym = symmetrize_derivatives(result)

Dict{Vector{WignerVar}, Any} with 3 entries:
  [∂αstar]     => (1//2)*αstar_sym
  [∂α, ∂αstar] => 1//4
  [∂α]         => (1//2)*α_sym

In [463]:
result = fokker_planck_dissipator([a(:a),a(:a)])
aval1_sym = symmetrize_derivatives(result)

Dict{Vector{WignerVar}, Any} with 5 entries:
  [∂αstar]             => -αstar_sym + α_sym*(αstar_sym^2)
  [∂α, ∂αstar, ∂αstar] => (1//8)*αstar_sym
  [∂α, ∂α, ∂αstar]     => (1//8)*α_sym
  [∂α, ∂αstar]         => (1//2)*(-(1//1) + (2//1)*α_sym*αstar_sym)
  [∂α]                 => -α_sym + (α_sym^2)*αstar_sym

In [464]:
result = fokker_planck_dissipator([ad(:a),ad(:a)])
aval1_sym = symmetrize_derivatives(result)

Dict{Vector{WignerVar}, Any} with 5 entries:
  [∂αstar]             => -αstar_sym - α_sym*(αstar_sym^2)
  [∂α, ∂αstar, ∂αstar] => (-1//8)*αstar_sym
  [∂α, ∂α, ∂αstar]     => (-1//8)*α_sym
  [∂α, ∂αstar]         => (1//2)*((1//1) + (2//1)*α_sym*αstar_sym)
  [∂α]                 => -α_sym - (α_sym^2)*αstar_sym

In [497]:
result = fokker_planck_dissipator([ad(:a),a(:a)])
aval1_sym = symmetrize_derivatives(result)

Dict{Vector{WignerVar}, Any} with 5 entries:
  [∂α, ∂α]         => (-1//2)*(α_sym^2)
  [∂αstar, ∂αstar] => (-1//2)*(αstar_sym^2)
  []               => -α_sym*αstar_sym
  [∂αstar]         => αstar_sym
  [∂α, ∂αstar]     => (1//2)*((1//4) + α_sym*αstar_sym)

In [504]:
result = fokker_planck_dissipator([ad(:a),a(:a)]; fac=1/2)
aval1_sym = symmetrize_derivatives(result)

Dict{Vector{WignerVar}, Any} with 5 entries:
  [∂α, ∂α]         => (-1//8)*(α_sym^2)
  [∂αstar, ∂αstar] => (-1//8)*(αstar_sym^2)
  []               => (-1//4)*α_sym*αstar_sym
  [∂αstar]         => (1//4)*αstar_sym
  [∂α, ∂αstar]     => (1//2)*((1//16) + (1//4)*α_sym*αstar_sym)

In [493]:
L_n = [a(:a),ad(:a),ad(:a)]
L_n2 = [a(:a), ad(:a), a(:a)]

# ========================================
# First term: L·ρ·L†
# ========================================
result1_left = operators_to_wigner([ad(:a),a(:a)], :right)      # L acts from left: L·ρ
result1_right = operators_to_wigner([a(:a)], :left)  # L† acts from right: ρ·L†
result1 = result1_left * result1_right*(-1/2)-1/2*operators_to_wigner([ad(:a)], :right)*operators_to_wigner([a(:a), ad(:a)], :left) 
terms1 = get_terms_with_coeffs(result1)

result_1st = Dict{Vector{WignerVar}, Rational{BigInt}}()
for term in terms1
    for (ops, coeff) in derivative_ordering_right_com(term)
        result_1st[ops] = get(result_1st, ops, Rational{BigInt}(0)) + coeff
    end
end

# ========================================
# Second term: -(1/2)L†L·ρ
# ========================================
result2 = operators_to_wigner(L_n, :right) * (1//4)+1/4*operators_to_wigner(L_n2, :right)
terms2 = get_terms_with_coeffs(result2)

result_2nd = Dict{Vector{WignerVar}, Rational{BigInt}}()
for term in terms2
    for (ops, coeff) in derivative_ordering_right_com(term)
        result_2nd[ops] = get(result_2nd, ops, Rational{BigInt}(0)) + coeff
    end
end

# ========================================
# Third term: -(1/2)ρ·L†L
# ========================================
result3 = operators_to_wigner(reverse(L_n) , :left) * (1//4)+1/4*operators_to_wigner(reverse(L_n2) , :left)
terms3 = get_terms_with_coeffs(result3)

result_3rd = Dict{Vector{WignerVar}, Rational{BigInt}}()
for term in terms3
    for (ops, coeff) in derivative_ordering_right_com(term)
        result_3rd[ops] = get(result_3rd, ops, Rational{BigInt}(0)) + coeff
    end
end

# ========================================
# Combine all three terms
# ========================================
result_merge = copy(result_1st)
for (ops, coeff) in result_2nd
    result_merge[ops] = get(result_merge, ops, Rational{BigInt}(0)) + coeff
end
for (ops, coeff) in result_3rd
    result_merge[ops] = get(result_merge, ops, Rational{BigInt}(0)) + coeff
end

# ========================================
# Final derivative ordering
# ========================================
terms_arranged = [(coeff, ops) for (ops, coeff) in result_merge]

result_all = Dict{Vector{WignerVar}, Rational{BigInt}}()
for term in terms_arranged
    for (ops, coeff) in derivative_ordering(term)
        result_all[ops] = get(result_all, ops, Rational{BigInt}(0)) + coeff
    end
end

res_add =  group_by_derivatives_symbolic(result_all)
result_add1 = fokker_planck_dissipator([ad(:a),a(:a)])
result_add2 = fokker_planck_dissipator([ad(:a)]; fac=1/2)


Dict{Vector{WignerVar}, Any} with 5 entries:
  [∂αstar, ∂α] => 1//32
  []           => 0
  [∂αstar]     => (-1//8)*αstar_sym
  [∂α, ∂αstar] => 3//32
  [∂α]         => (-1//8)*α_sym

In [494]:
result_new_d = copy(res_add)
for (ops, coeff) in result_add1
    result_new_d[ops] = get(result_merge, ops, Rational{BigInt}(0)) + coeff
end
for (ops, coeff) in result_add2
    result_new_d[ops] = get(result_merge, ops, Rational{BigInt}(0)) + coeff
end

In [495]:
result_new_d

Dict{Vector{WignerVar}, Any} with 15 entries:
  [∂α, ∂α]                 => (-1//2)*(α_sym^2)
  [∂αstar, ∂αstar]         => (-1//2)*(αstar_sym^2)
  [∂αstar]                 => (-1//8)*αstar_sym
  [∂α, ∂α, ∂αstar]         => -1//32
  [∂αstar, ∂α, ∂α]         => 1//32
  [∂α]                     => (-1//8)*α_sym
  [∂αstar, ∂αstar, ∂α]     => (-1//8)*αstar_sym
  [∂αstar, ∂α, ∂αstar]     => 0//1
  [∂α, ∂αstar, ∂αstar]     => -(1//16) + (1//8)*αstar_sym
  [∂α, ∂αstar, ∂α]         => -1//16
  [∂α, ∂αstar, ∂α, ∂αstar] => -1//16
  [∂αstar, ∂α]             => 1//32
  []                       => 0//1
  [∂α, ∂αstar]             => 3//32
  [∂α, ∂αstar, ∂αstar, ∂α] => 1//16

In [442]:
function fokker_planck_Heisenberg(L_ops::Vector{Operator}; mode::Symbol=:a)
    """
    Compute Fokker-Planck dissipator the Heisenberg or Commutator
    [L_ops, rho]
    
    Parameters:
    -----------
    L_ops : Vector of Operator defining the jump operator L
            Examples: [a(:a)] for D[a]
                     [ad(:a)] for D[a†]
                     [a(:a), a(:a)] for D[a²]
                     [ad(:a), ad(:a)] for D[(a†)²]
    mode : Symbol for the mode (default :a)
    
    Returns:
    --------
    Dictionary mapping WignerVar vectors to symbolic coefficients
    """
    

    result1 = operators_to_wigner(L_ops, :right) 
    terms1 = get_terms_with_coeffs(result1)
    
    result_1st = Dict{Vector{WignerVar}, Rational{BigInt}}()
    for term in terms1
        for (ops, coeff) in derivative_ordering_right_com(term)
            result_1st[ops] = get(result_1st, ops, Rational{BigInt}(0)) + coeff
        end
    end
    
    # ========================================
    # Second term: -(1/2)L†L·ρ
    # ========================================
    result2 = operators_to_wigner(reverse(L_ops), :left) * (-1)
    terms2 = get_terms_with_coeffs(result2)
    
    result_2nd = Dict{Vector{WignerVar}, Rational{BigInt}}()
    for term in terms2
        for (ops, coeff) in derivative_ordering_right_com(term)
            result_2nd[ops] = get(result_2nd, ops, Rational{BigInt}(0)) + coeff
        end
    end

    # ========================================
    # Combine all three terms
    # ========================================
    result_merge = copy(result_1st)
    for (ops, coeff) in result_2nd
        result_merge[ops] = get(result_merge, ops, Rational{BigInt}(0)) + coeff
    end
 
    # ========================================
    # Final derivative ordering
    # ========================================
    terms_arranged = [(coeff, ops) for (ops, coeff) in result_merge]
    
    result_all = Dict{Vector{WignerVar}, Rational{BigInt}}()
    for term in terms_arranged
        for (ops, coeff) in derivative_ordering(term)
            result_all[ops] = get(result_all, ops, Rational{BigInt}(0)) + coeff
        end
    end
    
    return group_by_derivatives_symbolic(result_all)
end



fokker_planck_Heisenberg (generic function with 1 method)

In [485]:
function fokker_planck_dissipator(L_ops::Vector{Operator}; mode::Symbol=:a, fac=1)
    """
    Compute Fokker-Planck dissipator D[L] for jump operator L
    
    D[L] = L·ρ·L† - (1/2){L†L, ρ}
         = L·ρ·L† - (1/2)L†L·ρ - (1/2)ρ·L†L
    
    Parameters:
    -----------
    L_ops : Vector of Operator defining the jump operator L
            Examples: [a(:a)] for D[a]
                     [ad(:a)] for D[a†]
                     [a(:a), a(:a)] for D[a²]
                     [ad(:a), ad(:a)] for D[(a†)²]
    mode : Symbol for the mode (default :a)
    
    Returns:
    --------
    Dictionary mapping WignerVar vectors to symbolic coefficients
    """
    
    # Construct L† by reversing and conjugating
    L_dag_ops = [is_creation(op) ? a(op.mode) : ad(op.mode) for op in reverse(L_ops)]
    
    # Construct L†L
    L_dag_L_ops = vcat(L_dag_ops, L_ops)
    ops_L_dag_L = vcat(L_ops,L_dag_ops)

    
    # ========================================
    # First term: L·ρ·L†
    # ========================================
    result1_left = operators_to_wigner(L_ops, :right)     # L acts from left: L·ρ
    result1_right = operators_to_wigner(reverse(L_dag_ops), :left)  # L† acts from right: ρ·L†
    result1 = result1_left * result1_right * fac * fac
    terms1 = get_terms_with_coeffs(result1)
    
    result_1st = Dict{Vector{WignerVar}, Rational{BigInt}}()
    for term in terms1
        for (ops, coeff) in derivative_ordering_right_com(term)
            result_1st[ops] = get(result_1st, ops, Rational{BigInt}(0)) + coeff
        end
    end
    
    # ========================================
    # Second term: -(1/2)L†L·ρ
    # ========================================
    result2 = operators_to_wigner(L_dag_L_ops, :right) * (-1//2) * fac * fac
    terms2 = get_terms_with_coeffs(result2)
    
    result_2nd = Dict{Vector{WignerVar}, Rational{BigInt}}()
    for term in terms2
        for (ops, coeff) in derivative_ordering_right_com(term)
            result_2nd[ops] = get(result_2nd, ops, Rational{BigInt}(0)) + coeff
        end
    end
    
    # ========================================
    # Third term: -(1/2)ρ·L†L
    # ========================================
    result3 = operators_to_wigner(ops_L_dag_L , :left) * (-1//2)*fac * fac
    terms3 = get_terms_with_coeffs(result3)
    
    result_3rd = Dict{Vector{WignerVar}, Rational{BigInt}}()
    for term in terms3
        for (ops, coeff) in derivative_ordering_right_com(term)
            result_3rd[ops] = get(result_3rd, ops, Rational{BigInt}(0)) + coeff
        end
    end
    
    # ========================================
    # Combine all three terms
    # ========================================
    result_merge = copy(result_1st)
    for (ops, coeff) in result_2nd
        result_merge[ops] = get(result_merge, ops, Rational{BigInt}(0)) + coeff
    end
    for (ops, coeff) in result_3rd
        result_merge[ops] = get(result_merge, ops, Rational{BigInt}(0)) + coeff
    end
    
    # ========================================
    # Final derivative ordering
    # ========================================
    terms_arranged = [(coeff, ops) for (ops, coeff) in result_merge]
    
    result_all = Dict{Vector{WignerVar}, Rational{BigInt}}()
    for term in terms_arranged
        for (ops, coeff) in derivative_ordering(term)
            result_all[ops] = get(result_all, ops, Rational{BigInt}(0)) + coeff
        end
    end
    
    return group_by_derivatives_symbolic(result_all)
end

# Usage examples:
# D[a]
result_Da = fokker_planck_dissipator([a(:a)])

# D[a†]
result_Dadag = fokker_planck_dissipator([ad(:a)])

# D[a²]
result_Da2 = fokker_planck_dissipator([a(:a), a(:a)])

# D[(a†)²]
result_Dadag2 = fokker_planck_dissipator([ad(:a), ad(:a)])

# D[a†a]
result_Dadaga = fokker_planck_dissipator([ad(:a), a(:a)])



Dict{Vector{WignerVar}, Any} with 14 entries:
  [∂α, ∂α]                 => (-1//2)*(α_sym^2)
  [∂αstar, ∂αstar]         => (-1//2)*(αstar_sym^2)
  [∂αstar]                 => αstar_sym
  [∂α, ∂α, ∂αstar]         => 0//1
  [∂α]                     => 0
  [∂αstar, ∂αstar, ∂α]     => (-1//8)*αstar_sym
  [∂αstar, ∂α, ∂αstar]     => 0//1
  [∂α, ∂αstar, ∂αstar]     => (1//8)*αstar_sym
  [∂α, ∂αstar, ∂α]         => 0//1
  [∂α, ∂αstar, ∂α, ∂αstar] => -1//16
  [∂αstar, ∂α]             => (1//4) + (1//4)*α_sym*αstar_sym
  []                       => -α_sym*αstar_sym
  [∂α, ∂αstar]             => (3//4)*α_sym*αstar_sym
  [∂α, ∂αstar, ∂αstar, ∂α] => 1//16

In [ ]:
function canonicalize_ops(ops::Vector{WignerVar})
    # Find split between variables and derivatives
    split_idx = length(ops)
    for i in 1:length(ops)
        if is_derivative(ops[i])
            split_idx = i - 1
            break
        end
    end
    
    vars = ops[1:split_idx]
    derivs = ops[split_idx+1:end]
    
    # Sort variables in canonical order (since they commute)
    # For example: sort by type, so all α come before all α*
    sorted_vars = sort(vars, by = op -> (op == wv_α() ? 0 : 1))
    
    # Keep derivatives in their exact order (they don't commute!)
    return vcat(sorted_vars, derivs)
end

# Then use it when building your results:
for term in terms1
    for (ops, coeff) in derivative_ordering(term)
        canonical_ops = canonicalize_ops(ops)
        result_1st[canonical_ops] = get(result_1st, canonical_ops, Rational{BigInt}(0)) + coeff
    end
end

















In [433]:
zz = [ad(:a), a(:a)]

2-element Vector{Operator}:
 Operator(:a, :creation)
 Operator(:a, :annihilation)

In [434]:
reverse(zz)

2-element Vector{Operator}:
 Operator(:a, :annihilation)
 Operator(:a, :creation)

In [111]:
ops = [a(:a)]
result01 = operators_to_wigner(ops, :right)
result02 = operators_to_wigner([ad(:a)], :left)
result1 = result01*result02
terms1 = get_terms_with_coeffs(result1);

result_1st = Dict{Vector{WignerVar}, Rational{BigInt}}()
for term in terms1
    merge!(+, result_1st , derivative_ordering(term))
end

result_1st

Dict{Vector{WignerVar}, Rational{BigInt}} with 4 entries:
  [∂αstar, ∂α]    => 1//4
  [αstar, αstar]  => 1
  [∂α, αstar]     => 1//2
  [∂αstar, αstar] => 1//2

In [105]:
key = [wv_dαstar()]  # or however you construct a WignerVar derivative
expr = group_all[key]

-(1//4)*α_sym + (1//4)*αstar_sym

In [106]:
key = [wv_dα()]  # or however you construct a WignerVar derivative
expr = group_all[key]

(1//2)*αstar_sym

In [91]:
resultnn = (αstar + R(1//2) * dα) * (α - R(1//2) * dαstar)*R(-1//2)

-1//2*α**α + 1//4*α**∂α* - 1//4*∂α*α + 1//8*∂α*∂α*

In [95]:
result

Dict{Vector{WignerVar}, Rational{BigInt}} with 8 entries:
  [∂αstar, ∂αstar] => 1//4
  [∂αstar, ∂α]     => 1//4
  []               => 0
  [αstar, αstar]   => 1
  [∂αstar, α]      => 0
  [αstar, α]       => -1
  [∂α, αstar]      => 1//2
  [∂αstar, αstar]  => 1//2

In [62]:
# First term of D[a^2]
ops = [a(:a), a(:a)]
result01 = operators_to_wigner(ops, :right)
result02 = operators_to_wigner([ad(:a), ad(:a)], :left)
result = result01*result02
terms1 = get_terms_with_coeffs(result);

result = Dict{Vector{WignerVar}, Rational{BigInt}}()
for term in terms1
    merge!(+, result, derivative_ordering(term))
end

In [66]:
@show result

result = Dict{Vector{WignerVar}, Rational{BigInt}}([∂αstar, ∂α, αstar] => 3//8, [∂αstar] => 0, [∂α] => -1//8, [∂αstar, αstar, αstar] => 1, [∂αstar, ∂αstar, αstar, αstar] => 1, [αstar] => -1, [∂αstar, ∂αstar, αstar] => 1//8, [∂αstar, ∂α] => 1//16, [∂α, ∂αstar, αstar] => 0, [αstar, α] => -1, [∂α, αstar] => -1//4, [∂α, α] => 0, [∂αstar, αstar] => -2, [∂α, ∂αstar, α] => 0, [α] => 0, [∂αstar, αstar, α] => 0, [∂α, α, αstar] => 1//2, [∂αstar, α, αstar] => 1//2, [] => 1//4, [∂αstar, ∂α, αstar, αstar] => 1//2, [α, αstar] => 1, [∂αstar, α] => -3//4, [∂αstar, ∂αstar, αstar, α] => 1//2, [∂α, ∂αstar] => -1//16)


Dict{Vector{WignerVar}, Rational{BigInt}} with 24 entries:
  [∂αstar, ∂α, αstar]            => 3//8
  [∂αstar]                       => 0
  [∂α]                           => -1//8
  [∂αstar, αstar, αstar]         => 1
  [∂αstar, ∂αstar, αstar, αstar] => 1
  [αstar]                        => -1
  [∂αstar, ∂αstar, αstar]        => 1//8
  [∂αstar, ∂α]                   => 1//16
  [∂α, ∂αstar, αstar]            => 0
  [αstar, α]                     => -1
  [∂α, αstar]                    => -1//4
  [∂α, α]                        => 0
  [∂αstar, αstar]                => -2
  [∂α, ∂αstar, α]                => 0
  [α]                            => 0
  [∂αstar, αstar, α]             => 0
  [∂α, α, αstar]                 => 1//2
  [∂αstar, α, αstar]             => 1//2
  []                             => 1//4
  ⋮                              => ⋮